## Install geemap

In [ ]:
#!pip install geemap

In [1]:
import ee
#import geemap

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=uyy4djYm3QgC1Tgq365zjCPFvxpa1xGygEA8bg6Z_Yk&tc=ERBKeAFdiSp-IGKvbMxJEbIMGMiGoBmpb1bGa80jtN4&cc=m4IgvBpSBLM8xrJ_HU955HrQIOEm8GqBhr4G43LfOoc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJr0NOIVm9KMLr5RVJ3OPpNFkS3-YtObnjNhD8iWxj1GnF65m9h9wKg

Successfully saved authorization token.


In [ ]:
#Map = geemap.Map()
#Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Extract Precipitation

### Constants:

In [4]:
# Ref: https:#developers.google.com/earth-engine/tutorials/community/ph-ug-temp

'Municipalities:'
# Import country boundaries feature collection.
dataset = ee.FeatureCollection('projects/ee-dsrestrepo/assets/mpio')
#Codes = dataset.aggregate_array('MPIOS')
Codes = {
    'Medellín':'05001',
    'Itagüí': '05360',
    'Barranquilla': '08001',
}

'Dates:'
# Define a date range of interest; here, a start date is defined and the end
start = ee.Date('2007-01-01')
dateRange = ee.DateRange(start, start.advance(13, 'year'))

'Satellite data:'
# Import LST image collection.
modis = ee.ImageCollection('MODIS/006/MOD11A1')
# Filter the LST collection to include only images intersecting the desired date range.
mod11a2 = modis.filterDate(dateRange)
# Select only the 1km day LST data band.
modLSTday = mod11a2.select('LST_Day_1km')
# Scale to Kelvin and convert to Celsius, set image acquisition time.

def func_rjp(img):
  return img \
    .multiply(0.02) \
    .subtract(273.15) \
    .copyProperties(img, ['system:time_start'])

modLSTc = modLSTday.map(func_rjp)

### Extract temperature for each municipality:

In [5]:
def func_hfv (image):
  date = image.date().format('yyyy-MM-dd')
  value = image.reduceRegion(
    reducer= ee.Reducer.mean(), 
    geometry= Municipality.geometry(), 
    scale= 1000).get('LST_Day_1km')
  return ee.Feature(None, {'value': value, 'date': date})

# Get time series data:
def get_time_series(Municipality):
  timeSeries = modLSTc.map(func_hfv)
  return timeSeries

In [6]:
for mun in Codes:
  code = Codes[mun]
  print(code)
  # Apply filter where municipalty code equals code.
  Municipality = dataset.filter(ee.Filter.eq('MPIOS', code))
  timeSeries = get_time_series(Municipality)

  task = ee.batch.Export.table.toDrive(** {
    'collection': timeSeries,
    'folder': 'GEE_temperature_TOP10',
    'description': 'temperature_' + code,
    'fileFormat': 'CSV',
    'selectors': ['value', 'date']
  })
  
  task.start()

print("Done!")

05001
ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.map",
    "arguments": {
      "baseAlgorithm": {
        "functionDefinitionValue": {
          "argumentNames": [
            "_MAPPING_VAR_0_0"
          ],
          "body": {
            "functionInvocationValue": {
              "functionName": "Feature",
              "arguments": {
                "metadata": {
                  "dictionaryValue": {
                    "values": {
                      "date": {
                        "functionInvocationValue": {
                          "functionName": "Date.format",
                          "arguments": {
                            "date": {
                              "functionInvocationValue": {
                                "functionName": "Image.date",
                                "arguments": {
                                  "image": {
                                    "argumentReference": "_MAPPING_VAR_0_0"
        